In [1]:
%%capture
!pip install mljar-supervised

In [2]:
%cd /kaggle/working

/kaggle/working


In [3]:
import pandas as pd
df = pd.read_csv('../input/tencent1/11.csv')
df

,status,play number,score,year,topic
0,45.0,13.3y,7.9,2021,0
1,31.0,7339.1w,7.7,2021,0
2,48.0,4.0y,9.2,2021,0
3,36.0,19.2y,9.4,2021,0
4,34.0,1.8y,7.7,2021,0
...,...,...,...,...,...
2947,NaN,3.8w,8.6,2011,15
2948,NaN,1.6w,8.5,2011,15
2949,NaN,1.4w,7.5,2011,15
2950,NaN,NaN,7.5,2011,15


In [4]:
df1 = df.dropna(subset=['play number']).reset_index(drop=True)
df1

,status,play number,score,year,topic
0,45.0,13.3y,7.9,2021,0
1,31.0,7339.1w,7.7,2021,0
2,48.0,4.0y,9.2,2021,0
3,36.0,19.2y,9.4,2021,0
4,34.0,1.8y,7.7,2021,0
...,...,...,...,...,...
2936,NaN,6.4w,8.8,2011,15
2937,NaN,3.6w,8.5,2011,15
2938,NaN,3.8w,8.6,2011,15
2939,NaN,1.6w,8.5,2011,15


In [5]:
def clean_number(s):
    if s[-1] == 'w':
        return float(s[:-1])
    if s[-1] == 'y':
        return float(s[:-1]) * 1e4
    return float(s)
df1['play number'] = df1['play number'].apply(clean_number)
df1

,status,play number,score,year,topic
0,45.0,133000.0,7.9,2021,0
1,31.0,7339.1,7.7,2021,0
2,48.0,40000.0,9.2,2021,0
3,36.0,192000.0,9.4,2021,0
4,34.0,18000.0,7.7,2021,0
...,...,...,...,...,...
2936,NaN,6.4,8.8,2011,15
2937,NaN,3.6,8.5,2011,15
2938,NaN,3.8,8.6,2011,15
2939,NaN,1.6,8.5,2011,15


In [6]:
!git clone https://github.com/mljar/mljar-supervised.git
%cd mljar-supervised
!python setup.py install

Cloning into 'mljar-supervised'...
remote: Enumerating objects: 7576, done.
remote: Counting objects: 100% (1268/1268), done.
remote: Compressing objects: 100% (444/444), done.
remote: Total 7576 (delta 927), reused 1084 (delta 820), pack-reused 6308
Receiving objects: 100% (7576/7576), 8.27 MiB | 5.39 MiB/s, done.
Resolving deltas: 100% (5788/5788), done.
/kaggle/working/mljar-supervised
running install
running bdist_egg
running egg_info
creating mljar_supervised.egg-info
writing mljar_supervised.egg-info/PKG-INFO
writing dependency_links to mljar_supervised.egg-info/dependency_links.txt
writing requirements to mljar_supervised.egg-info/requires.txt
writing top-level names to mljar_supervised.egg-info/top_level.txt
writing manifest file 'mljar_supervised.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mljar_supervised.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
runn

In [7]:
df1['play cat'] = 0
for i in range(len(df1)):
    n = df1.loc[i, 'play number']
    if n < 8429:
        df1.loc[i, 'play cat'] = 0
    elif n > 174291:
        df1.loc[i, 'play cat'] = 2
    else:
        df1.loc[i, 'play cat'] = 1

In [8]:
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df1[['status', 'score', 'year', 'topic']], df1["play cat"], test_size=0.1
)
cls = LGBMClassifier()
cls.fit(X_train, y_train)
predictions = cls.predict(X_test)

In [9]:
predictions

array([0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 2, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 2, 0, 0, 2, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 2, 0,
       2, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 1, 2, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1])

In [10]:
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average='micro')

0.7864406779661017

In [11]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[105,  23,   0],
       [ 20, 115,   5],
       [  3,  12,  12]])

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.7864406779661017